In [1]:
import os
import requests

# Función para descargar archivos necesarios del repo de PyTorch
def download_file(url):
    local_filename = url.split('/')[-1]
    if not os.path.exists(local_filename):
        print(f"Descargando {local_filename}...")
        r = requests.get(url)
        with open(local_filename, 'wb') as f:
            f.write(r.content)
    else:
        print(f"{local_filename} ya existe.")

base_url = "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/"
files = ["engine.py", "utils.py", "coco_utils.py", "coco_eval.py", "transforms.py"]

for f in files:
    download_file(base_url + f)

print("¡Archivos de ayuda descargados!")

Descargando engine.py...
Descargando utils.py...
Descargando coco_utils.py...
Descargando coco_eval.py...
Descargando transforms.py...
¡Archivos de ayuda descargados!


In [9]:
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from PIL import Image
import os
import numpy as np

# IMPORTANTE: Cambia esto a la ruta de tu carpeta
# Estructura esperada:
# root/images/imagen1.jpg
# root/masks/imagen1.png
ROOT_DIR = r"C:\Users\josei\personal-projects\scd-ml\data\HAM10000" 

# Importamos las utilidades que descargamos en el Bloque 1
from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [ ]:
class HAM10000Dataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # Ordenamos para asegurar que imagen X coincida con mascara X
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))

    def __getitem__(self, idx):
        # Cargar rutas
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])
        
        # Abrir imagen
        img = Image.open(img_path).convert("RGB")
        
        # Abrir máscara (convertir a numpy array)
        # Nota: Asumimos que la máscara es binaria (0 fondo, 255 lesión)
        mask = Image.open(mask_path)
        mask = np.array(mask)
        
        # Obtener ids de objetos (asumimos 1 lesión por foto, o varias)
        obj_ids = np.unique(mask)
        # El primer id es el fondo (0), lo quitamos
        obj_ids = obj_ids[1:]

        # Crear máscaras booleanas para cada objeto
        masks = mask == obj_ids[:, None, None]

        # Calcular Bounding Boxes
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        if num_objs == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            
        labels = torch.ones((num_objs,), dtype=torch.int64) # Label 1 = Lesión
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [12]:
def get_model_instance_segmentation(num_classes):
    # 1. Cargar modelo pre-entrenado en COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # 2. Reemplazar la cabeza del detector de cajas (FastRCNN)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 3. Reemplazar la cabeza del detector de máscaras (MaskRCNN)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

# Función auxiliar para transformaciones básicas
import torchvision.transforms.functional as F

# --- AGREGA ESTA CLASE MANUALMENTE PARA CORREGIR EL ERROR ---
class ToTensor(torch.nn.Module):
    def forward(self, image, target):
        image = F.to_tensor(image)
        return image, target
    
def get_transform(train):
    transforms = []
    # Ahora usamos nuestra clase ToTensor manual, así que no necesitamos llamar a T.ToTensor
    transforms.append(ToTensor()) 
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [13]:
# 1. Configuración de dispositivo
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Usando dispositivo: {device}")

# 2. Clases: Fondo (0) + Lesión (1) = 2 clases
num_classes = 2

# 3. Cargar datos
# Usa tu dataset definido en el Bloque 3
dataset = HAM10000Dataset(ROOT_DIR, get_transform(train=True))
dataset_test = HAM10000Dataset(ROOT_DIR, get_transform(train=False))

# Dividir dataset (Ejemplo: 50 train, resto test para prueba rápida)
# AJUSTA ESTOS NUMEROS SEGÚN EL TAMAÑO DE TU DATASET
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# 4. Crear DataLoaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=0, # num_workers=0 para evitar errores en Windows
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

# 5. Instanciar Modelo
model = get_model_instance_segmentation(num_classes)
model.to(device)

# 6. Optimizador y Learning Rate
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Scheduler para bajar el LR
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Usando dispositivo: cuda


c:\Users\josei\personal-projects\scd-ml\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\josei\personal-projects\scd-ml\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
num_epochs = 10  # Puedes subir esto a 20 o 30 si tienes tiempo

for epoch in range(num_epochs):
    # Entrenar por una época
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    
    # Actualizar learning rate
    lr_scheduler.step()
    
    # Evaluar en el set de prueba (opcional, tarda un poco)
    evaluate(model, data_loader_test, device=device)

print("¡Entrenamiento finalizado!")

# Guardar el modelo
torch.save(model.state_dict(), "maskrcnn_ham10000.pth")
print("Modelo guardado como maskrcnn_ham10000.pth")

c:\Users\josei\personal-projects\scd-ml\notebooks\engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [   0/4983]  eta: 4:40:11  lr: 0.000010  loss: 5.0694 (5.0694)  loss_classifier: 0.6635 (0.6635)  loss_box_reg: 0.1833 (0.1833)  loss_mask: 4.1969 (4.1969)  loss_objectness: 0.0176 (0.0176)  loss_rpn_box_reg: 0.0081 (0.0081)  time: 3.3738  data: 0.2256  max mem: 1880
Epoch: [0]  [  10/4983]  eta: 1:02:09  lr: 0.000060  loss: 2.9691 (3.3407)  loss_classifier: 0.5634 (0.5528)  loss_box_reg: 0.1324 (0.1329)  loss_mask: 2.1827 (2.6231)  loss_objectness: 0.0114 (0.0162)  loss_rpn_box_reg: 0.0144 (0.0157)  time: 0.7499  data: 0.1229  max mem: 2052
Epoch: [0]  [  20/4983]  eta: 0:51:25  lr: 0.000110  loss: 2.1677 (2.4966)  loss_classifier: 0.3797 (0.3990)  loss_box_reg: 0.1305 (0.1229)  loss_mask: 1.6175 (1.9391)  loss_objectness: 0.0150 (0.0202)  loss_rpn_box_reg: 0.0134 (0.0155)  time: 0.4840  data: 0.1105  max mem: 2052
Epoch: [0]  [  30/4983]  eta: 0:47:20  lr: 0.000160  loss: 0.9276 (1.9538)  loss_classifier: 0.1252 (0.3067)  loss_box_reg: 0.1008 (0.1187)  loss_mask: 0.6640 (

IndexError: too many indices for tensor of dimension 1

In [ ]:
import matplotlib.pyplot as plt

# Poner modelo en modo evaluación
model.eval()

# Tomar una imagen del test set
img, _ = dataset_test[0]

# Hacer predicción
with torch.no_grad():
    prediction = model([img.to(device)])

# Convertir imagen a formato visible (CPU)
img_show = img.mul(255).permute(1, 2, 0).byte().numpy()
plt.figure(figsize=(10,10))
plt.imshow(img_show)

# Obtener máscaras predichas (con confianza > 0.5)
masks = prediction[0]['masks']
scores = prediction[0]['scores']
mask_threshold = 0.5

# Superponer la primera máscara detectada con alta confianza
if len(masks) > 0 and scores[0] > mask_threshold:
    mask_show = masks[0, 0].mul(255).byte().cpu().numpy()
    plt.imshow(mask_show, alpha=0.5, cmap='jet') # Alpha da transparencia
    print(f"Lesión detectada con confianza: {scores[0]:.2f}")
else:
    print("No se detectaron lesiones con suficiente confianza.")

plt.show()